## Building a Machine Learning Model to Predict cLogD
In this notebook we will use the descriptors we calculated using the **calc_descriptors.py** script build a model to predict cLogD.  First we'll import the necessary libraries. 

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor, Booster
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import chembl_downloader
import pystow
from constants import SFI_MODULE

Enable Pandas progress_apply

In [2]:
tqdm.pandas()

In [3]:
chembl_version = chembl_downloader.latest()
print(f"The latest version of ChEMBL is {chembl_version}")

[ypo] Missing information in OLS


The latest version of ChEMBL is 31


In [4]:
SFI_MODULE = SFI_MODULE.module(chembl_version)

### 1. Read the Descriptors
Read the data generated by **dask_descriptors.py**. 

In [5]:
%%time
df = pd.read_pickle(SFI_MODULE.join(name="logd_descriptors.pkl"))

CPU times: user 9.35 s, sys: 3.49 s, total: 12.8 s
Wall time: 13.6 s


Let's see how much data we read. 

In [6]:
df.shape

(2304875, 4)

As a sanity check, let's look at the first few rows.

In [7]:
df.head()

,smiles,name,logd,desc
0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl,1,2.69,"[2.151684657491086, -2.0923071597340894, 2.216..."
1,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1,2,1.82,"[2.1309007703070395, -2.0857480375944135, 2.17..."
2,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1,3,2.64,"[2.1708426473880804, -2.1841553872256485, 2.29..."
3,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1,4,1.97,"[2.093212484023919, -2.0505383418976266, 2.126..."
4,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1,5,2.57,"[2.129504518042437, -2.0858337096441177, 2.182..."


Drop any data with nulls. 

In [8]:
df.dropna(inplace=True)

Let's see how much data remains.  In practice, we'd try to figure out why some molecules didn't generate descriptors.  In this case, we still have more that 2 million records, so we're fine. 

In [9]:
df.shape

(2247904, 4)

Convert the descriptors from lists to numpy arrays. In retrospect, I may have been able to do this when I generated, the descriptors.  Oh well, not a big deal. 

### 2. Build the ML Model
Split the data into training and test sets. 

In [10]:
%%time
train, test = train_test_split(df)

CPU times: user 1.19 s, sys: 23.1 ms, total: 1.21 s
Wall time: 1.21 s


Use [np.stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html) to convert the descriptors to an appropriate format for ML model building. 

In [11]:
%%time
X_train = np.stack(train.desc.values)
y_train = train.logd.values
X_test = np.stack(test.desc.values)
y_test = test.logd.values

CPU times: user 5.97 s, sys: 6.66 s, total: 12.6 s
Wall time: 13.5 s


Build an ML model.  Wow, LightGBM is fast! 

In [12]:
%%time
lgbm = LGBMRegressor()
lgbm.fit(X_train,y_train)

CPU times: user 5min 46s, sys: 9.03 s, total: 5min 55s
Wall time: 25.5 s


LGBMRegressor()

Predict on the test set.

In [13]:
%%time
pred = lgbm.predict(X_test)

CPU times: user 8.92 s, sys: 1.94 s, total: 10.9 s
Wall time: 759 ms


### 3. Test the ML Model
Calculate $R^2$

In [14]:
r2_score(y_test,pred)

0.8986728679269809

Calculate RMSE

In [15]:
mean_squared_error(y_test,pred, squared=False)

0.791424981719083

### 4. Save the ML Model
Save the model to disk.

In [16]:
model_file = SFI_MODULE.join(name="model.txt")
lgbm.booster_.save_model(model_file)

Read the model from disk.

In [17]:
mdl = Booster(model_file=model_file)

Predict with the saved model and calculate $R^2$

In [18]:
pred = mdl.predict(X_test)

In [19]:
mean_squared_error(y_test,pred, squared=False)

0.791424981719083